In [44]:
# Step 1 : Load data into ChromaDB using Google Embedding Model
import os
import chromadb
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

# 1. Load the PDF
loader = PyPDFLoader(r"C:\Applied_GenAI\Course-4-AGS Advanced Generative AI Building LLM Applications\Assessment\AI_Powered_HR_Assistance\nestle_hr_policy.pdf")
pages = loader.load()

all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# 2. Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunks = text_splitter.split_text(joined_page_text)

# 3. Configure Gemini API key
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# 4. Initialize the Google embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

# 5. Load the Chroma database from disk
persist_directory = r"C:\Applied_GenAI\Jupyter_Notebooks\basic_langchain\chroma_storage"
chroma_db = Chroma(persist_directory=persist_directory, 
                   embedding_function=embeddings,
                   collection_name="gemini_chroma_demo")

# 6. Get the collection from the Chroma database
collection = chroma_db.get()

# 7. If the collection is empty, create a new one
if len(collection['ids']) == 0:
    # Create a new Chroma database from the documents
    chroma_db = Chroma.from_texts(
        texts=chunks, 
        embedding=embeddings, 
        persist_directory=persist_directory,
        collection_name="gemini_chroma_demo"
    )
chroma_db.persist()

error uploading: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)')))


In [40]:
# Step 2: Combine a retriver and QA Chain
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# 1. Initialize the Google Generative AI Model
model = ChatGoogleGenerativeAI(model="gemini-pro",
                               google_api_key=api_key,
                               temperature=0.3,
                               convert_system_message_to_human=True
                              ) 

# 2. Create prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, say that you don't know, don't try to make up an answer. 
Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question} 
Helpful Answer:"""

qa_prompt = PromptTemplate(input_variables=["context","question"]
                                 , template=template, 
                                ) 
# 3. load a chain for question-answering
qa_chain = RetrievalQA.from_chain_type(llm=model
                                       , retriever= chroma_db.as_retriever()
                                       , return_source_documents=True
                                       , chain_type="stuff"
                                       , chain_type_kwargs={"prompt": qa_prompt} 
                                      )

input_variables=['context', 'question'] input_types={} partial_variables={} template='Use the following pieces of context to answer the question at the end. \nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer. \nKeep the answer as concise as possible. \nAlways say "thanks for asking!" at the end of the answer. \n{context}\nQuestion: {question} \nHelpful Answer:'


C:\Users\rachchh\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [42]:
# Step 3: Method to ask a question to gemini
def chat_with_gemini(usermessage):
    result=qa_chain(usermessage)
    print(result)
    return result["result"]

In [43]:
# Step 4: Integration with gradio
import gradio as gr

demo = gr.Interface(fn=chat_with_gemini,
                    inputs=gr.Textbox(label= "What can I help with?"),
                    outputs=gr.Textbox(label= "AI-generated response"),
                    title="Nestle AI-Powered HR Assistant")
# To create a public link, set `share=True` in `launch()`
demo.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


C:\Users\rachchh\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'query': 'What is the prime responsiblity of Line Manager?', 'result': 'The prime responsibility of Line Manager is to build and sustain an environment where people have a sense of personal commitment to their work and give their best to ensure our Company’s success. \nThanks for asking!', 'source_documents': [Document(metadata={}, page_content='Jean-Marc Duvoisin\nDeputy Executive Vice President\n Introduction The Nestlé Human Resources Policy\n2\nLine managers have the prime responsibility for \nbuilding and sustaining an environment where \npeople have a sense of personal commitment \nto their work and give their best to ensure our \nCompany’s success. They care for and develop \nthe leaders of tomorrow.\nLine managers decide on all people matters \nunder their influence, within the boundaries set'), Document(metadata={}, page_content='values and principles, or programmes conducted \nby our strategic learning partners. \n Training and learning\nAt Nestlé, a high performance culture

C:\Users\rachchh\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'query': 'What is the prime responsiblity of Development Manager?', 'result': "I'm sorry, but the provided context does not mention anything about the prime responsibility of a Development Manager, so I cannot answer this question. \nThanks for asking!", 'source_documents': [Document(metadata={}, page_content='Jean-Marc Duvoisin\nDeputy Executive Vice President\n Introduction The Nestlé Human Resources Policy\n2\nLine managers have the prime responsibility for \nbuilding and sustaining an environment where \npeople have a sense of personal commitment \nto their work and give their best to ensure our \nCompany’s success. They care for and develop \nthe leaders of tomorrow.\nLine managers decide on all people matters \nunder their influence, within the boundaries set'), Document(metadata={}, page_content='responsible for guiding and coaching employees \nto succeed in their current positions.  \nNestlé employees understand the importance \nof continuous improvement, as well as sharing \n

...

{'query': 'What is the prime responsiblity of HR department?', 'result': 'I cannot answer this question because the provided context does not mention the prime responsibility of the HR department. Thanks for asking!', 'source_documents': [Document(metadata={}, page_content='by the policies and principles, acting as the final \ndecision makers.\nThe Human Resources (HR) structure \nenables and empowers them in establishing \nbusiness needs and their corresponding people \nrequirements. \nTherefore, the mission of HR managers and \ntheir teams is to provide professional guidance \nto line managers aiming to deliver superior \nbusiness results by optimising the performance \nof our people, while ensuring exemplary working \nconditions.'), Document(metadata={}, page_content='Jean-Marc Duvoisin\nDeputy Executive Vice President\n Introduction The Nestlé Human Resources Policy\n2\nLine managers have the prime responsibility for \nbuilding and sustaining an environment where \npeople have a se

C:\Users\rachchh\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'query': 'What is the prime responsiblity of Nestle?', 'result': 'I cannot answer this question from the provided context. \nThanks for asking!', 'source_documents': [Document(metadata={}, page_content='Employment and working conditions  The Nestlé Human Resources Policy\n4\nLearning is part of the Company culture.\nEmployees at all levels are systematically \nencouraged to consider how they upgrade their \nknowledge and skills.\nThe Company determines training and deve-\nlopment priorities. The responsibility for turning \nthese into actions is shared between employees, \nline managers and the Human Resources. \nExperience and on-the-job training are the \nprimary source of learning. Managers are'), Document(metadata={}, page_content='community involvement. \nThose with line management responsibilities \nare required to take personal ownership of safety \nand health within their area of responsibility and \nare encouraged to develop their capability in this \narea.\nNestlé’s commitme

C:\Users\rachchh\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'query': 'What is  Nestle?', 'result': 'Nestle is a company that produces food and beverages. \nThanks for asking!', 'source_documents': [Document(metadata={}, page_content='Nestec Ltd., Corporate Identity & Design,  \nVevey, Switzerland\nProduction\nbrain’print GmbH, Switzerland\nPaper\nThis report is printed on BVS, a paper produced  \nfrom well-managed forests and other controlled sources  \ncertified by the Forest Stewardship Council (FSC). The Nestlé Human Resources Policy\n1\nAt Nestlé, we recognize that our employees \nare the key to our success and nothing can be \nachieved without their engagement. \nThis document encompasses the guidelines'), Document(metadata={}, page_content='which constitute a solid basis for effective Human \nResources Management throughout the Nestlé \nGroup around the world. It explains to all Nestlé \nemployees the vision and mission of the Human \nResources function and illustrates every aspect of \nthe Nestlé employee lifecycle. \nThe Nestlé Managem